In [1]:
# NEEDED PACKAGES -> no visualization!!

using StochasticDelayDiffEq
using SpecialFunctions
using Distributions
using SimulatedAnnealingABC
using Distances
using DataFrames
using FFTW
using CSV
using XLSX
using ThreadPinning

In [2]:
include("./SDDESolarDynamo.jl")
using .SDDESolarDynamo

In [3]:
# Distance function in the sABC algorithm
function f_dist(θ::Vector{Float64}; type::Int64 = 1, indices::Union{Vector{Int64}, StepRange{Int64, Int64}} = 1:6:120, fourier_data::Vector{Float64})
  sol = bfield(θ, Tsim)
  
  simulated_data = sol[1,:]
  simulated_data = simulated_data .^ 2
  fourier_stats = reduced_fourier_spectrum(simulated_data, indices)

  rho = [euclidean(fourier_stats[i], fourier_data[i]) for i in 1:length(fourier_stats)]
  return rho
end

# function for the summary statistics
function reduced_fourier_spectrum(u::Vector{Float64}, indices::Union{Vector{Int64}, StepRange{Int64, Int64}} = 1:6:120)
  fourier_transform = abs.(fft(u))
  return fourier_transform[indices]
end

reduced_fourier_spectrum (generic function with 2 methods)

In [4]:
# FUNCTIONS NEEDED FOR SAVING THE RESULTS OF A SIMULATION

# function to create a new directory for each simulations, in order to store the needed files
function create_directory()
  base_path = pwd()
  base_path = joinpath(base_path, "Simulations")
  i = 1
  dir_name = "Real $i"
  dir_path = joinpath(base_path, dir_name)
  
  while isdir(dir_path)
    i += 1
    dir_name = "Real $i"
    dir_path = joinpath(base_path, dir_name)
  end
  
  mkpath(dir_path)
  println("Directory created at: $dir_path")
  cd(dir_path)
end

# function to save the prior as a string
function get_prior_string(prior)
  parts = []
  for d in prior.dists
    if isa(d, Uniform)
      push!(parts, "Uniform($(minimum(d)), $(maximum(d)))")
    else
      error("Unsupported distribution type: $(typeof(d))")
    end
  end
  
  return "product_distribution(" * join(parts, ", ") * ")"
end

# function to save the sabc parameters
function save_sabc_params(prior, n_particles::Int, n_simulation::Int, v::Float64, type::Int, indices::Union{Vector{Int}, StepRange{Int64, Int64}})
  curr_path = pwd()
  filename = "sabc_params.csv"
  path = joinpath(curr_path, filename)
    
  sabc_params = DataFrame(
    Parameter = ["prior", "n_particles", "n_simulation", "v", "type", "indices"],
    Value = [get_prior_string(prior), n_particles, n_simulation, v, type, string(indices)]
  )
    
 CSV.write(filename, sabc_params) 
 println("Parameters saved to: $path")
end

# Function to save the result object of a sABC algorithm
function save_result(result::SimulatedAnnealingABC.SABCresult{Vector{Float64}, Float64})
  curr_path = pwd()
  filenames = ["eps_hist.csv", "u_hist.csv", "rho_hist.csv"]
  variables = [result.state.ϵ_history, result.state.u_history, result.state.ρ_history]

  for (filename, variable) in zip(filenames, variables)
    labels = string.(1:size(variable, 1))
    path = joinpath(curr_path, filename)
    CSV.write(path, DataFrame(variable, labels))
    println("$filename data saved to: $path")
  end

  filename = "pop.csv"
  path = joinpath(curr_path, filename)

  param_samples = hcat(result.population...)

  posterior_params = DataFrame(
    N_value = param_samples[1, :],
    T_value = param_samples[2, :],
    tau_value = param_samples[3, :],
    sigma_value = param_samples[4, :],
    Bmax_value = param_samples[5, :]
  )

  CSV.write(path, posterior_params)
  println("Posterior parameters saved to: $path")

  filename = "rho.csv"
  path = joinpath(curr_path, filename)

  rho = result.ρ

  rho_values = DataFrame(rho, [:ss1, :ss2, :ss3])

  CSV.write(path, rho_values)
  println("Rho values saved to: $path")
end

save_result (generic function with 1 method)

In [5]:
# THREADS SETTINGS AND INFO

ThreadPinning.pinthreads(:cores)
ThreadPinning.threadinfo()


System: 8 cores (no SMT), 8 sockets, 1 NUMA domains

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 

# = Julia thread, | = Socket seperator

Julia threads: 8
├ Occupied CPU-threads: 8
└ Mapping (Thread => CPUID): 1 => 0, 2 => 1, 3 => 2, 4 => 3, 5 => 4, ...


In [6]:
# EXTRACTING OPEN MAGNETIC FLUX AND SUNSPOT NUMBER RECORDS FROM XLSX FILE

# Define DataFrame object
data = DataFrame(
  year = Int[],
  open_magn_flux = Float64[],
  open_magn_flux_err = Float64[],
  ssa_open_magn_flux = Float64[],
  sunspots_num = Float64[],
  sunspots_err = Float64[],
  ssa_sunspots = Float64[]
)

# Open file and for each row write data into the DataFrame
XLSX.openxlsx("SN Usoskin Brehm.xlsx") do file
  sheet = file["Data"] 

  for row in XLSX.eachrow(sheet)
    if isa(row[2], Number)
      push!(data, (
        year = row[2],
        open_magn_flux = row[3],
        open_magn_flux_err = row[4],
        ssa_open_magn_flux = row[5],
        sunspots_num = row[7],
        sunspots_err = row[8],
        ssa_sunspots = row[9]
      ))
    end
  end
end

# Creation of the summary statistics for the real data
u = data.open_magn_flux
indices = [1, 2, 85]

sim_ss = reduced_fourier_spectrum(u, indices)

3-element Vector{Float64}:
 3993.8700000000003
  649.4478744969342
  274.8631648867697

In [7]:
# DIRECTORY MANAGING

# Current directory
initial_directory = pwd()

# New directory
create_directory()

# NB: After "create_directory", we move to the new directory.
# DON'T RUN THIS AGAIN, wait for the simulation to finish!!!! If you made errors,
# eliminate the Simulations/Simulation i directory and then rerun everything

Directory created at: /home/ubuntu/LCP_B/Project/Simulations/Real 9


In [8]:
# SIMULATION PARAMETERS MANAGING

# Parameters that can be tuned for new simulations
prior = product_distribution(Uniform(5, 15), Uniform(0.1, 10.0), Uniform(0.1, 6.0), Uniform(0.01, 0.3), Uniform(1, 15))
n_particles = 1000
n_simulation = 10000000
v = 1.0
type = 1

# Writing on file "sabc_params.csv" of the values set in this cell.
save_sabc_params(prior, n_particles, n_simulation, v, type, indices)

Parameters saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 9/sabc_params.csv


In [9]:
# SIMULATION

# Conditions
tmin = data.year[1]; tmax = data.year[end]
Tsim = tmax

# Actual usage of the sABC algorithm
result = sabc(f_dist, prior;
              n_particles = n_particles, 
              n_simulation = n_simulation,
              v = v,
              type = type,
              indices = indices,
              fourier_data = sim_ss)

# Display of the summary of the results
display(result)

# Saving the results to the files: "eps_hist.csv", "u_hist.csv", "rho_hist.csv", "pop.csv", "rho.csv".
#save_result(result)

┌ Info: Preparing to run SABC algorithm: 'single-epsilon'
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:188
┌ Info: Using threads: 8 
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:199
┌ Info: Set BLAS threads = 1 
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:202
┌ Info: Set 'pinthreads(:cores)' for optimal multi-threading performance
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:204
┌ Info: Initializing population...
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:223
┌ Info: Initial resampling (δ = 0.1) - ESS = 996.6799495718244 
└ @ SimulatedAnnealingABC /home/ubuntu/.julia/packages/SimulatedAnnealingABC/e8QsC/src/SimulatedAnnealingABC.jl:277
┌ Info: Population

Approximate posterior sample with 1000 particles:
  - simulations used: 10000000
  - average transformed distance: 0.0001785
  - ϵ: [1.003e-5]
  - population resampling: 10
  - acceptance rate: 0.002007
The sample can be accessed with the field `population`.
The history of ϵ can be accessed with the field `state.ϵ_history`.
 -------------------------------------- 


In [10]:
save_result(result)

eps_hist.csv data saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 9/eps_hist.csv
u_hist.csv data saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 9/u_hist.csv
rho_hist.csv data saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 9/rho_hist.csv
Posterior parameters saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 9/pop.csv
Rho values saved to: /home/ubuntu/LCP_B/Project/Simulations/Real 9/rho.csv


In [11]:
# DIRECTORY MANAGING

# WE go back to the initial directory
cd(initial_directory)
pwd()

"/home/ubuntu/LCP_B/Project"

In [12]:
rho = result.ρ

1000×3 Matrix{Float64}:
 0.0367841   0.00610747   0.0065129
 0.00481952  0.00287875   0.00436299
 0.00974315  0.0111278    0.0176448
 0.0171469   0.0184687    0.00682275
 0.0207351   0.00723567   0.0130263
 0.00572407  0.00212297   0.0129065
 0.00310571  0.0195331    0.0105904
 0.0198678   0.0183426    0.010909
 0.00320793  0.00867594   0.0368647
 0.0240655   0.00783285   0.000554024
 ⋮                        
 0.010252    0.00839131   0.0189056
 0.020576    0.0104372    0.0222773
 0.0306596   0.0226583    0.00480644
 0.0025033   0.0194599    0.00798541
 0.00134421  0.0123293    0.0108947
 0.00117684  0.00415272   0.0285427
 0.00372859  0.000507827  0.00113087
 0.0230464   0.00492986   0.0175802
 0.00402466  0.0123165    0.00895995

In [13]:
df = DataFrame(rho, :auto)

df_squared = DataFrame()

# Iterate over each column and compute the square of each element
for col in names(df)
    df_squared[!, col] = df[!, col] .^ 2
end

# Now `df_squared` contains the square of each entry in the original DataFrame
println(df_squared)

1000×3 DataFrame
  Row │ x1           x2           x3          
      │ Float64      Float64      Float64     
──────┼───────────────────────────────────────
    1 │ 0.00135307   3.73011e-5   4.24178e-5
    2 │ 2.32278e-5   8.28719e-6   1.90357e-5
    3 │ 9.4929e-5    0.000123827  0.00031134
    4 │ 0.000294017  0.000341092  4.65499e-5
    5 │ 0.000429944  5.23549e-5   0.000169683
    6 │ 3.2765e-5    4.50699e-6   0.000166578
    7 │ 9.64544e-6   0.000381543  0.000112157
    8 │ 0.000394729  0.000336453  0.000119007
    9 │ 1.02908e-5   7.5272e-5    0.00135901
   10 │ 0.00057915   6.13536e-5   3.06943e-7
   11 │ 0.000140858  0.000134476  0.00136622
   12 │ 1.98546e-5   0.0001834    7.6305e-5
   13 │ 0.000326474  7.33255e-5   0.000705379
   14 │ 0.000212211  0.000339514  0.00016998
   15 │ 0.000650195  5.7574e-5    0.000507672
   16 │ 1.26627e-5   0.000306545  5.40061e-5
   17 │ 0.000470249  0.000162817  1.04013e-5
   18 │ 8.63932e-8   0.000305863  0.000527933
   19 │ 0.00132248   0.000

In [14]:
row_sums = Vector{Float64}(undef, size(df, 1))

# Compute the sum of each row and store it in `row_sums`
for (i, row) in enumerate(eachrow(df_squared))
    row_sums[i] = sum(row)
end

# Now `row_sums` contains the sum of each row
k = 5  # Number of minimum values you want to find
new_indices = partialsortperm(row_sums, 1:k)  # Indices of the 5 smallest values
min_values = row_sums[new_indices]  # The 5 smallest values

println("Minimum values: ", min_values)
println("Indices of minimum values: ", new_indices)

Minimum values: [7.732158959117682e-6, 8.250341963912977e-6, 8.907091411764405e-6, 9.362361240342463e-6, 1.5421516111925658e-5]
Indices of minimum values: [396, 792, 567, 672, 189]


In [15]:
param_samples = hcat(result.population...)

posterior_params = DataFrame(
  N_value = param_samples[1, :],
  T_value = param_samples[2, :],
  tau_value = param_samples[3, :],
  sigma_value = param_samples[4, :],
  Bmax_value = param_samples[5, :]
)

Row,N_value,T_value,tau_value,sigma_value,Bmax_value
,Float64,Float64,Float64,Float64,Float64
1,5.82817,2.92891,3.23948,0.2408,11.3371
2,13.054,0.667939,1.43245,0.221795,8.62489
3,11.2361,6.49013,0.256082,0.273218,8.66622
4,6.91158,4.44783,2.84198,0.271324,8.61027
5,11.6703,0.673968,3.65502,0.248884,7.32132
6,12.0091,0.731001,3.63217,0.208078,7.29045
7,14.1089,5.9189,0.633797,0.279483,8.86048
8,7.92672,5.56572,2.75894,0.259552,8.30501
9,5.9546,0.693657,1.94922,0.29926,8.07215


In [16]:
new_indices

5-element view(::Vector{Int64}, 1:5) with eltype Int64:
 396
 792
 567
 672
 189

In [17]:
df[new_indices, :]

Row,x1,x2,x3
,Float64,Float64,Float64
1,5.36808e-5,0.00025986,0.00276799
2,0.00164495,0.0019261,0.00135448
3,0.00291321,0.000126977,0.000635752
4,0.00176541,0.00211605,0.00132966
5,0.00032243,0.00116691,0.00373576


In [18]:
best_particles = posterior_params[new_indices, :]

Row,N_value,T_value,tau_value,sigma_value,Bmax_value
,Float64,Float64,Float64,Float64,Float64
1,5.27127,2.03644,3.68423,0.254666,11.0162
2,14.3088,0.558498,1.399,0.139888,7.4803
3,5.0946,6.51697,1.32416,0.234768,9.88649
4,5.23806,2.62187,0.350597,0.29252,9.15741
5,13.4664,0.406672,0.684498,0.0432887,8.33523
